# install

In [133]:
from tqdm.auto import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib


In [134]:
!pip install chromedriver-autoinstaller


In [135]:
# 크롬 드라이버 자동 실행 코드
from selenium import webdriver
import chromedriver_autoinstaller

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인

try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe')

driver.implicitly_wait(10)


In [136]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [137]:
options = webdriver.ChromeOptions() # 크롬 옵션 객체 생성
user_agent = "Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2049.0  Safari/537.36"
options.add_argument('user-agent=' + user_agent)
options.add_argument('headless') # headless 모드 설정
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument("--mute-audio") #mute
options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
options.add_argument("--start-maximized")

In [138]:
import os
import pandas as pd
import winsound

# 링크 파일 불러오기 
ytb = pd.read_csv('youtube_link.csv')
ytb_link = ytb.link.to_list()

# 자동화 구간
for i in ytb_link :
    
    # 드라이버 불러오기
    driver = webdriver.Chrome('chromedriver.exe', options= options)
    driver.get(i)

    # 스크롤 다운
    time.sleep(1.5) 
    endkey = 4 # 90~120개 / 늘릴때 마다 30개
    while endkey:
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(0.3)
        endkey -= 1
    
    # 채널 요소들 가져오기    
    channel_name = driver.find_element_by_xpath('//*[@id="text-container"]').text # 채널 명
    subscribe = driver.find_element_by_css_selector('#subscriber-count').text   # 구독자수
    channel_name = re.sub('[=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]', '', channel_name) # 채널 이름에 특수문자 제거
    # print(channel_name,subscribe)

    # bs4 실행    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    video_list0 = soup.find('div', {'id': 'contents'})
    video_list2 = video_list0.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

    base_url = 'https://www.youtube.com'
    video_url = []

    # 반복문을 실행시켜 비디오의 주소를 video_url에 넣는다.
    for i in range(len(video_list2)):
        url = base_url+video_list2[i].find('a',{'id':'thumbnail'})['href']
        video_url.append(url)

    driver.quit() # 종료

    if subscribe :
        channel = channel_name + ' - ' + subscribe
    else :
        channel = channel_name


    directory = f'data/{channel}/subtitle'
    if not os.path.exists(directory):
        os.makedirs(directory)

    print(channel, len(video_url))

    ytb_info(video_url, channel)
    print()
    winsound.PlaySound('sound.wav', winsound.SND_FILENAME)



FileNotFoundError: [Errno 2] No such file or directory: 'youtube_link.csv'

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib.request
import time
from IPython.display import display
import warnings
warnings.filterwarnings(action='ignore')

path = 'C:/Users/ad/sources/Final-project/102/chromedriver.exe' #'path 값은 이용자의 브라우저 드라이버가 설치된 장소로 지정'
def get_video():
    feature = input('검색어를 입력하시오 : ')
    
    driver =  webdriver.Chrome(path)
    driver.get('https://www.youtube.com')
    
    n = 3
    while n > 0:
        print('웹페이지를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
    
    src = driver.find_element_by_xpath('//*[@id="search"]')
    src.send_keys(feature)
    src.send_keys(Keys.RETURN)
    
    n = 2
    while n > 0:
        print('검색 결과를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
        
    print('데이터 수집 중입니다....')
    
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    df_title = []
    df_link = []
    df_writer = []
    df_view = []
    df_date = []
    
    for i in range(len(soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated'))):
        title = soup.find_all('a', {'id' : 'video-title'})[i].text.replace('\n', '')
        link = 'https://www.youtube.com/' + soup.find_all('a', {'id' : 'video-title'})[i]['href']
        writer = soup.find_all('ytd-channel-name', 'long-byline style-scope ytd-video-renderer')[i].text.replace('\n', '').split(' ')[0]
        view = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[3]
        date = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[4]
    
        df_title.append(title)
        df_link.append(link)
        df_writer.append(writer)
        df_view.append(view)
        df_date.append(date)
          
    df_just_video = pd.DataFrame(columns=['영상제목', '채널명', '영상url', '조회수', '영상등록날짜'])

    df_just_video['영상제목'] = df_title
    df_just_video['채널명'] = df_writer
    df_just_video['영상url'] = df_link
    df_just_video['조회수'] = df_view
    df_just_video['영상등록날짜'] = df_date
          
    df_just_video.to_csv('../data/df_just_video.csv', encoding='utf-8-sig', index=False)
    
    driver.close()
          
    result = input('데이터프레임 저장이 완료되었습니다! 데이터프레임을 조회하시겠습니까? (y/n)')
    if result == 'y':
        display(df_just_video)
        question = input('원하는 영상을 재생하시겠습니까? (y/n)')
        if question == 'y':
            button = int(input('재생하고자 하는 영상의 번호(출력된 표 가장 왼쪽의 번호)를 입력해주세요.'))
            driver = webdriver.Chrome(path)
            driver.get(df_just_video['영상url'][button])
        else:
            return '프로그램을 종료합니다.'
    else:
        return '프로그램을 종료합니다.'

In [ ]:
# 접속
driver =  webdriver.Chrome(path)
driver.get('https://www.youtube.com/results?search_query=%EC%9E%AC%ED%85%8C%ED%81%AC&sp=EgIoAQ%253D%253D')

for n in range(100):
    driver.execute_script('window.scrollBy(0,1000)')
    # 다음 작업을 코드적으로 강제 대기 시킨것
    # 데이터 추출시에는 selenium에서 지원하는 waits 기법을 사용하면 좋다
    time.sleep(2)

In [ ]:
# 영상 데이터 한줄 : 
# ytd-video-renderer
# 영상 메타 정보 :
# #video-title

# 영상 갯수 확인인 셈
videos = driver.find_elements_by_tag_name('ytd-video-renderer')
len( videos ), type(videos)

(412, list)

In [ ]:
#driver.find_element_by_css_selector
for video in videos[:2]:
    # 영상의 타이틀
    print( video.find_element_by_id('video-title').get_attribute('title') )
    # 썸네임
    print( video.find_element_by_tag_name('img').get_attribute('src') )
    # 링크
    print( video.find_element_by_id('video-title').get_attribute('href') )
    # 작성자
    print( video.find_element_by_css_selector('#channel-info>#channel-name>#container>#text-container>#text >a').text )
    # 조회수 : metadata-line, # 작성시간
    for span in video.find_elements_by_css_selector('#metadata-line>span'):
        print( span.text)
    '''
    [
      { 'title':'xx', 'thum':'xx', 'link':'xx', 'author':'xx', 'views':x, 'time':''}
      ,{},{},...
    ]
    '''
    

2천만원으로 30억을 번 92년생의 재테크 방법 (부자되는 세상)
https://i.ytimg.com/vi/fgHb4YJFoWI/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCKVxAhPi3iVL_pRKr3jAQAGA0LsA
https://www.youtube.com/watch?v=fgHb4YJFoWI
김작가 TV
조회수 46만회
2개월 전
절대 돈 안 잃는 재테크 3가지, 부자들은 꼭 하는 투자방법
https://i.ytimg.com/vi/hZQs2_Gn5Io/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBbSrm2WC5GLs3-i504UG-YFhPH6A
https://www.youtube.com/watch?v=hZQs2_Gn5Io
마케팅웨이 최준호 - 온라인 브랜딩 전문가
조회수 1만회
1개월 전


In [ ]:
# 전체 들고오기
results = []
for video in videos:#[:2]:    
    dic = {
        'title':video.find_element_by_id('video-title').get_attribute('title')
        ,'thum':video.find_element_by_tag_name('img').get_attribute('src')
        ,'link':video.find_element_by_id('video-title').get_attribute('href')
        ,'author':video.find_element_by_css_selector('#channel-info>#channel-name>#container>#text-container>#text >a').text
        ,'views':video.find_elements_by_css_selector('#metadata-line>span')[0].text
        ,'time':video.find_elements_by_css_selector('#metadata-line>span')[1].text
    }
    results.append( dic )
results

[{'title': '2천만원으로 30억을 번 92년생의 재테크 방법 (부자되는 세상)',
  'thum': 'https://i.ytimg.com/vi/fgHb4YJFoWI/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCKVxAhPi3iVL_pRKr3jAQAGA0LsA',
  'link': 'https://www.youtube.com/watch?v=fgHb4YJFoWI',
  'author': '김작가 TV',
  'views': '조회수 46만회',
  'time': '2개월 전'},
 {'title': '절대 돈 안 잃는 재테크 3가지, 부자들은 꼭 하는 투자방법',
  'thum': 'https://i.ytimg.com/vi/hZQs2_Gn5Io/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBbSrm2WC5GLs3-i504UG-YFhPH6A',
  'link': 'https://www.youtube.com/watch?v=hZQs2_Gn5Io',
  'author': '마케팅웨이 최준호 - 온라인 브랜딩 전문가',
  'views': '조회수 1만회',
  'time': '1개월 전'},
 {'title': '재테크책 150권 중 5권 추천한다면?(90년생 20억자산 Ep.04)',
  'thum': 'https://i.ytimg.com/vi/8zi2RHhAaco/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLAS-wtk7J-NZcWlfxAUKksLC7NVew',
  'link': 'https://www.youtube.com/watch?v=8zi2RHhAaco',
  'author': '절약왕TV',
  'views': '조회수 4.5만회',
  'time': '4개월 전'},
 {'tit

In [ ]:
links = [ ]
for link in results :
    links.append(link['link'])

links

['https://www.youtube.com/watch?v=fgHb4YJFoWI',
 'https://www.youtube.com/watch?v=hZQs2_Gn5Io',
 'https://www.youtube.com/watch?v=8zi2RHhAaco',
 'https://www.youtube.com/watch?v=M8_fSjRBT54',
 'https://www.youtube.com/watch?v=gDEiTx8-Qko',
 'https://www.youtube.com/watch?v=M5riCjuMUco',
 'https://www.youtube.com/watch?v=LTGqoa91MQg',
 'https://www.youtube.com/watch?v=EOqs9yZLtjI',
 'https://www.youtube.com/watch?v=90fhxh0rB-A',
 'https://www.youtube.com/watch?v=mIUxSJn5eRU',
 'https://www.youtube.com/watch?v=QgBTEaPoL-g',
 'https://www.youtube.com/watch?v=7qskAqeagXg',
 'https://www.youtube.com/watch?v=UElb_gDeGgQ',
 'https://www.youtube.com/watch?v=7w7Xk_-Y5aA',
 'https://www.youtube.com/watch?v=qMUlHdvntJQ',
 'https://www.youtube.com/watch?v=fnDWhfufPCE',
 'https://www.youtube.com/watch?v=-B2GQVWYY6A',
 'https://www.youtube.com/watch?v=9TC4j07vlFk',
 'https://www.youtube.com/watch?v=X4YY6dWfkOA',
 'https://www.youtube.com/watch?v=pQCRlUtK5WY',
 'https://www.youtube.com/watch?v=01xlCM

In [ ]:
!pip install  youtube_transcript_api
!pip install  konlpy
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [139]:
from youtube_transcript_api import YouTubeTranscriptApi
from konlpy.tag import Kkma
from pykospacing import Spacing


def ytb_subtitle(links, title):
    try:
        code = links.split('=')[1]
        srt = YouTubeTranscriptApi.get_transcript(f"{code}", languages=['ko']) #한글로, 딕셔너리 구조

        text = ''

        for i in range(len(srt)):
            text += srt[i]['text'] + ''

        text_ = text.replace(' ','')

        #문장 분리 / kss 사용해도 무방
        kkma = Kkma()

        text_sentences = kkma.sentences(text_)

        #종결 단어
        lst = ['죠','다','요','시오', '습니까','십니까','됩니까','옵니까','뭡니까',]

        df = pd.read_csv('not_verb.csv',encoding='utf-8')
        not_verb = df.stop.to_list()

        #단어 단위로 끊기
        text_all = ' '.join(text_sentences).split(' ')

        for n in range(len(text_all)) :
            i = text_all[n]
            if len(i) == 1 : #한글자일 경우 추가로 작업x
                continue

            else :
                for j in lst : #종결 단어
                    #질문형
                    if j in lst[4:]:
                        i += '?'

                    #명령형                
                    elif j == '시오':
                        i += '!'

                    #마침표    
                    else :
                        if i in not_verb : #특정 단어 제외
                            continue
                        else :
                            if j == i[len(i)-1] : #종결
                                text_all[n] += '.'


        spacing = Spacing()
        text_all_in_one = ' '.join(text_all)

        text_split = spacing(text_all_in_one.replace(' ','')).split('.')
        text2one= []
        for t in text_split:
            text2one.append(t.lstrip())

        w = '. '.join(text2one)

        f = open(f'data/{channel}/subtitle/{title}.txt','w',encoding='utf-8')
        f.write(w)
        f.close()
        print('O')
    except:
        print('X')

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [132]:
!pip install  youtube_transcript_api
!pip install  konlpy
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\ad\appdata\local\temp\pip-req-build-xqx4_3j2
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit 1732408600abf4a106413d05e3ed1d7cc59dc3a6
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.4
    Uninstalling tensorflow-2.6.4:
      Successfully uninstalled tensorflow-2.6.4


  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\ad\AppData\Local\Temp\pip-req-build-xqx4_3j2'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pykospacing 0.5 requires tensorflow==2.6.4, but you have tensorflow 2.7.2 which is incompatible.


In [33]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\ad\AppData\Local\Temp\pip-req-build-tjcy40ai'



  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\ad\appdata\local\temp\pip-req-build-tjcy40ai
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b28bd05611db76c60d309f227a7748681dcf7d5e
  Using cached tensorflow-2.6.4-cp39-cp39-win_amd64.whl (429.2 MB)
  Using cached h5py-3.1.0-cp39-cp39-win_amd64.whl (2.7 MB)
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorboard-2.6.0-py3-none-any.whl (5.6 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.46.3-cp39-cp39-win_amd64.whl (3.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Using

In [45]:
import os
import pandas as pd
import winsound
import time
from bs4 import BeautifulSoup
import re
from youtube_transcript_api import YouTubeTranscriptApi
from konlpy.tag import Kkma
from pykospacing import Spacing


def ytb_subtitle(start_url, title):
    try:
        code = start_url.split('=')[1]
        srt = YouTubeTranscriptApi.get_transcript(f"{code}", languages=['ko'])#한글로, 딕셔너리 구조

        text = ''
        
        for i in range(len(srt)):
            text += srt[i]['text'] + ' '

        text_ = text.replace(' ','')

        #문장 분리 / kss 사용해도 무방
        kkma = Kkma()

        text_sentences = kkma.sentences(text_)

        #종결 단어
        lst =   ['죠','다','요','시오', '습니까','십니까','됩니까','옵니까','뭡니까',]

        df = pd.read_csv('not_verb.csv',encoding='utf-8')
        not_verb = df.stop.to_list()

        #단어 단위로 끊기
        text_all = ' '.join(text_sentences).split(' ')

        for n in range(len(text_all)) :
            i= text_all[n]
            if len(i) == 1 : #한글자일 경우 추가로 작업x
                continue

            else :
                for j in lst : #종결 단어
                        #질문형
                    if j in lst[4:]:
                        i += '?'

                        #명령형                
                    elif j == '시오' :
                        i += '!'
                    #마침표    
                    else :
                        if i in not_verb : #특정 단어 제외
                            continue
                        else :
                            if j == i[len(i)-1] : #종결
                                    text_all[n] += '.'


            spacing = Spacing()
            text_all_in_one = ' '.join(text_all)

            text_split = spacing(text_all_in_one.replace(' ','')).split('.')
            text2one= []
            for t in text_split:
                text2one.append(t.lstrip())

            w = '. '.join(text2one)

            f = open(f'data/{channel}/subtitle/{title}.txt','w',encoding='utf-8')
            f.write(w)
            f.close()
            print('O')
    except:
        print('X')




TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [44]:
ytb_subtitle('https://www.youtube.com/watch?v=hZQs2_Gn5Io', '절대 돈 안 잃는 재테크 3가지, 부자들은 꼭 하는 투자방법')

NameError: name 'ytb_subtitle' is not defined

In [41]:
import winsound as sd
from bs4 import BeautifulSoup
import pyautogui
import pandas as pd
import re

def beepsound():
    fr = 2000 # rnage : 37~32767
    du = 1000 # 1000 ms == 1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

def ytb_info(video_url,channel) :
    print(f'{channel}', '크롤링 시작')
    driver = webdriver.Chrome('chromedriver.exe', options = options)
    # new_data ={'data : '', 'title': '', 'view': '', 'comment': '', 'like' : '', 'dislike':''}
    
    count =1
    
    # 데이터 넣을 리스트
    data_list = []
    title_list = []
    view_list =[]
    like_list = []
    dislike_list = []
    comment_list = []
    
    try :
            #각 채널별 영상으로 크롤링
        for i in range(len(video_url)) :
            start_url = video_url[i]
            print(start_url, end = '/')
            driver.get(start_url)
            driver.implicitly_wait(1.5)
            
            body = driver.finad_element_by_tag_name('body')
            
            # 댓글 null 값 방지
            num_of_pagedowns = 1
            while num_of_pagedowns :
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.5)
                num_of_pagedowns -= 1
                driver.implicitly_wait(1.5)
                
                body = driver.find_element_by_tag_name('body')
                
                #댓글 null 값 방지
                num_of_pagedowns :
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.5)
                num_of_pagedowns -= 1
                driver.implicitly_wait(1)
                        
            # 크롤링 요소
            try :
                info = driver.find_element_by_css_selector('.style-scope ytd-video-primary-info-renderer').text.split('/n')
                
                if '인기 급상스 동영상' in info[0] :
                    info.pop(0)
                elif '#' in info[0].split(''[0]) :
                    info.pop(0)
                
                title = info[0]
                divide = info[1].replace('조회수, ','').replace(',','').split('회')
                view = divide[0]
                date = divide[1].replace('','')
                like = info[2]
                dislike = info[3]
                
                try :
                    comment = driver.find_element_by_css_selector('#count > yt-formatted-string > span:nth-child(2)').text.replace(',','')
                except :
                    comment = '댓글x'
                
                # 리스트에 추가
                title_list.append(title)
                view_list.append(view)
                data_list.append(date)
                like_list.append(like)
                dislike_list.append(dislike)
                comment_list.append(comment)
                
                # 크롤링 정보 저장
                new_data = {'data' : date_list
                            , 'title':title_list
                            , 'view': view_list
                            , 'coment': comment_list
                            , 'like':like_list
                            , 'dislike' : dislike_list}
                df = pd.DataFrame(new_data)
                df.to_csv(f'data/{channel}/-{channel}.csv', encoding ='utf-8-sig')
            except :
                continue
            # print(title, view, date, like, dislike, comment)
            
            num_of_pagedowns = 1
            while num_of_pagedowns :
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.5)
                num_of_pagedonwns -= 1
            # 페이지 다운
            last_page_height = driver.execute_script("return document.documentElement.scrollHeight")
            
            While True :
                driver.execute_script("windowscrollTo(0, document.documentElement.scrollHeight);")
                time.sleep(0.5)
                new_page_height = driver.execute_script("return document.document.scrollHeight")
                
                if new_page_height == last_page_height :
                    break
                last_page_height = new_page_height
                # driver.implicitly-wait(1)
                time.sleep(0.75)
                
            time.sleep(0.5)
            
            # 댓글 크롤링
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            
            users = soup.select("div#header-author > h3 > #author-text > span")
            comments = soup.select("yt-formatted-string#content-text")
            
            user_list = []
            review_list=[]
            
            for i in range(len(users)) :
                str_tmp = str(users[i].text)
                str_tmp =str_tmp.replace('\n', '')
                str_tmp =str_tmp.replace('\t', '')
                str_tmp =str_tmp.replace('              ','')
                str_tmp =str_tmp.replace('            ','')
                user_list.append(str_tmp)
                
                str_tmp = str(comments[i].text)
                str_tmp = str_tmp.replace('\n', '')
                str_tmp = str_tmp.replace('\t', '')
                str_tmp = str_tmp.replace('            ', '')
                
                review_list.append(str_tmp)
            
            # 댓글 추가
            pd_data = {"ID" : user_list, "Comment" : review_list}
            youtube_pd = pd.DataFrame(pd_data)
            
            title = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…《\》]','', title)
            youtube_pd.to_csv(f"data/{channel}/{title}.csv", encoding = 'utf-8-sig')#,index_col = False)
            print('ㅁ',end='')
            
            # 자막 추출
            ytb_subtitle(start_url, title)
            
            # 광고 끄기
            if count :
                # time.sleeep(1)
                try : 
                    driver.implicitly_wait(0.5)
                    driver.find_element_by_css_selector("#main >div>ytd-button-renderer").click()
                    count -=1
                except:
                    continue
                
        except :
                driver.quit()
                beepsound()
        driver.quit()
        beepsound()   
        


            
            
                
                
                
                       
                
    

SyntaxError: invalid syntax (Temp/ipykernel_8984/1725859883.py, line 48)